In [52]:
import azureml.core

print("This notebook was created using version 1.0.23 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.23 of the Azure ML SDK
You are currently using version 1.0.39 of the Azure ML SDK


In [53]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="942813ed-26a2-4790-9097-d4be7560fce0")
resource_group = os.getenv("RESOURCE_GROUP", default="ICSE-tutorial")
workspace_name = os.getenv("WORKSPACE_NAME", default="workspace_fresher")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

In [54]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible.")

Workspace configuration succeeded.


### Create compute resources for your training experiments

Many of the sample notebooks use Azure ML managed compute (AmlCompute) to train models using a dynamically scalable pool of compute. In this section you will create default compute clusters for use by the other notebooks and any other operations you choose.

To create a cluster, you need to specify a compute configuration that specifies the type of machine to be used and the scalability behaviors.  Then you choose a name for the cluster that is unique within the workspace that can be used to address the cluster later.

The cluster parameters are:
* vm_size - this describes the virtual machine type and size used in the cluster.  All machines in the cluster are the same type.  You can get the list of vm sizes available in your region by using the CLI command

```shell
az vm list-skus -o tsv
```
* min_nodes - this sets the minimum size of the cluster.  If you set the minimum to 0 the cluster will shut down all nodes while note in use.  Setting this number to a value higher than 0 will allow for faster start-up times, but you will also be billed when the cluster is not in use.
* max_nodes - this sets the maximum size of the cluster.  Setting this to a larger number allows for more concurrency and a greater distributed processing of scale-out jobs.


To create a **CPU** cluster now, run the cell below. The autoscale settings mean that the cluster will scale down to 0 nodes when inactive and up to 4 nodes when busy.

In [55]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpucluster


In [56]:
import azureml.dataprep as dprep

# Load data
Download two different NYC Taxi data sets into dataflow objects. These datasets contain slightly different fields. The method auto_read_file() automatically recognizes the input file type.

In [ ]:
from IPython.display import display

eclipse_2_0_raw = dprep.r

In [14]:
from IPython.display import display
dataset_root = "https://dprepdata.blob.core.windows.net/demo"

green_path = "/".join([dataset_root, "green-small/*"])
yellow_path = "/".join([dataset_root, "yellow-small/*"])

green_df_raw = dprep.read_csv(path=green_path, header=dprep.PromoteHeadersMode.GROUPED)
# auto_read_file automatically identifies and parses the file type, which is useful when you don't know the file type.
yellow_df_raw = dprep.auto_read_file(path=yellow_path)

display(green_df_raw.head(5))
display(yellow_df_raw.head(5))


,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type,Column21,Column22
0,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,1,0,0,0,0,1,...,0,0,0,0,,21.25,2,,,
2,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,1,0,0,0,0,2,...,0,0.5,0,0,,75,2,,,
3,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,5,0,0,0,0,1,...,0.1,0,0,1,,2.1,2,,,
4,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,1,0,0,0,0,1,...,0,0,0,0,,3.25,2,,,


,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,,,-73.993803,40.695922000000003,CASH,8.9000000000000004,0.5,,0,0,9.4000000000000004
2,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,,,-73.955849999999998,40.768030000000003,Credit,12.1,0.5,,2,0,14.6
3,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,,,-73.869983000000005,40.770225000000003,Credit,23.699999999999999,0,,4.7400000000000002,0,28.440000000000001
4,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,,,-73.996557999999993,40.731848999999997,CREDIT,14.9,0.5,,3.0499999999999998,0,18.449999999999999


# Cleanse data
Now you populate some variables with shortcut transforms to apply to all dataflows. The drop_if_all_null variable is used to delete records where all fields are null. The useful_columns variable holds an array of column descriptions that are kept in each dataflow.

In [15]:
all_columns = dprep.ColumnSelector(term=".*", use_regex=True)
drop_if_all_null = [all_columns, dprep.ColumnRelationship(dprep.ColumnRelationship.ALL)]
useful_columns = [
    "cost", "distance", "dropoff_datetime", "dropoff_latitude", "dropoff_longitude",
    "passengers", "pickup_datetime", "pickup_latitude", "pickup_longitude", "store_forward", "vendor"
]

In [17]:
green_df = (green_df_raw
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "VendorID": "vendor",
        "lpep_pickup_datetime": "pickup_datetime",
        "Lpep_dropoff_datetime": "dropoff_datetime",
        "lpep_dropoff_datetime": "dropoff_datetime",
        "Store_and_fwd_flag": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Pickup_longitude": "pickup_longitude",
        "Pickup_latitude": "pickup_latitude",
        "Dropoff_longitude": "dropoff_longitude",
        "Dropoff_latitude": "dropoff_latitude",
        "Passenger_count": "passengers",
        "Fare_amount": "cost",
        "Trip_distance": "distance"
     })
    .keep_columns(columns=useful_columns))
green_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,0,0,0,0,1,.00,21.25
1,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,0,0,0,0,2,.00,74.5
2,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,0,0,0,0,1,.00,1
3,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,0,0,0,0,1,.00,3.25
4,2,2013-08-01 11:51:45,2013-08-01 12:03:52,N,0,0,0,0,1,.00,8.5


In [18]:

yellow_df = (yellow_df_raw
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "vendor_name": "vendor",
        "VendorID": "vendor",
        "vendor_id": "vendor",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "tpep_pickup_datetime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "store_and_forward": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Passenger_Count": "passengers",
        "passenger_count": "passengers",
        "Fare_Amt": "cost",
        "fare_amount": "cost",
        "Trip_Distance": "distance",
        "trip_distance": "distance"
    })
    .keep_columns(columns=useful_columns))
yellow_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,passengers,distance,pickup_longitude,pickup_latitude,store_forward,dropoff_longitude,dropoff_latitude,cost
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,None,-73.993803,40.695922000000003,8.9000000000000004
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,None,-73.955849999999998,40.768030000000003,12.1
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,None,-73.869983000000005,40.770225000000003,23.699999999999999
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,None,-73.996557999999993,40.731848999999997,14.9
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40000000000000002,-74.001580000000004,40.719382000000003,None,-74.008377999999993,40.720350000000003,3.7000000000000002


In [19]:
combined_df = green_df.append_rows([yellow_df])


In [29]:

decimal_type = dprep.TypeConverter(data_type=dprep.FieldType.DECIMAL)
combined_df = combined_df.set_column_types(type_conversions={
    "pickup_longitude": decimal_type,
    "pickup_latitude": decimal_type,
    "dropoff_longitude": decimal_type,
    "dropoff_latitude": decimal_type
})
combined_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
pickup_longitude,FieldType.DECIMAL,-115.179337,0.000000,7821.0,0.0,7821.0,0.0,0.0,0.0,,-80.020035,-73.963275,-73.963704,-73.948626,-73.922876,-73.846188,0.000000,0.000000,0.000000,-68.898728,18.682200,349.024613,3.396883,9.659230
pickup_latitude,FieldType.DECIMAL,0.000000,40.919121,7821.0,0.0,7821.0,0.0,0.0,0.0,,0.000000,40.685431,40.683272,40.721164,40.756150,40.803989,40.849260,40.870135,40.891918,37.972373,10.285086,105.782988,-3.418050,9.691664
dropoff_longitude,FieldType.DECIMAL,-115.179337,0.000000,7821.0,0.0,7821.0,0.0,0.0,0.0,,-85.253196,-73.985200,-73.985796,-73.957136,-73.930112,-73.866722,0.000000,0.000000,0.000000,-68.961163,18.586449,345.456096,3.420567,9.823180
dropoff_latitude,FieldType.DECIMAL,0.000000,41.008934,7821.0,0.0,7821.0,0.0,0.0,0.0,,0.000000,40.665640,40.661916,40.718315,40.756607,40.783887,40.852088,40.879613,40.925072,37.999084,10.230188,104.656755,-3.442054,9.856544


In [30]:

latlong_filtered_df = (combined_df
    .drop_nulls(
        columns=["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],
        column_relationship=dprep.ColumnRelationship(dprep.ColumnRelationship.ANY)
    )
    .filter(dprep.f_and(
        dprep.col("pickup_longitude") <= -73.72,
        dprep.col("pickup_longitude") >= -74.09,
        dprep.col("pickup_latitude") <= 40.88,
        dprep.col("pickup_latitude") >= 40.53,
        dprep.col("dropoff_longitude") <= -73.72,
        dprep.col("dropoff_longitude") >= -74.09,
        dprep.col("dropoff_latitude") <= 40.88,
        dprep.col("dropoff_latitude") >= 40.53
    )))
latlong_filtered_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
pickup_longitude,FieldType.DECIMAL,-74.078156,-73.736481,7158.0,0.0,7158.0,0.0,0.0,0.0,,-74.078005,-73.963899,-73.963992,-73.949486,-73.929364,-73.868618,-73.830461,-73.824454,-73.769637,-73.914784,0.049049,0.002406,0.403432,-0.616452
pickup_latitude,FieldType.DECIMAL,40.575485,40.879852,7158.0,0.0,7158.0,0.0,0.0,0.0,,40.633389,40.713281,40.713110,40.721378,40.758137,40.804996,40.848658,40.867524,40.877764,40.765038,0.048107,0.002314,0.237329,-0.577294
dropoff_longitude,FieldType.DECIMAL,-74.085747,-73.720871,7158.0,0.0,7158.0,0.0,0.0,0.0,,-74.078163,-73.986191,-73.986471,-73.960094,-73.937219,-73.885699,-73.815741,-73.776719,-73.733574,-73.921367,0.056023,0.003139,0.660521,0.029285
dropoff_latitude,FieldType.DECIMAL,40.583530,40.879734,7158.0,0.0,7158.0,0.0,0.0,0.0,,40.600511,40.695779,40.695739,40.727841,40.758163,40.787331,40.850385,40.867809,40.878691,40.759402,0.050253,0.002525,0.048660,-0.013012


In [31]:
latlong_filtered_df.keep_columns(columns='store_forward').get_profile()


,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
store_forward,FieldType.STRING,N,Y,7158.0,198.0,6960.0,0.027661,0.0,0.0,3,,,,,,,,,,,,,,


In [32]:
replaced_stfor_vals_df = latlong_filtered_df.replace(columns="store_forward", find="0", replace_with="N").fill_nulls("store_forward", "N")

In [33]:
replaced_distance_vals_df = replaced_stfor_vals_df.replace(columns="distance", find=".00", replace_with=0).fill_nulls("distance", 0)
replaced_distance_vals_df = replaced_distance_vals_df.to_number(["distance"])

In [34]:
time_split_df = (replaced_distance_vals_df
    .split_column_by_example(source_column="pickup_datetime")
    .split_column_by_example(source_column="dropoff_datetime"))
time_split_df.head(5)

,vendor,pickup_datetime,pickup_datetime_1,pickup_datetime_2,dropoff_datetime,dropoff_datetime_1,dropoff_datetime_2,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [35]:
renamed_col_df = (time_split_df
    .rename_columns(column_pairs={
        "pickup_datetime_1": "pickup_date",
        "pickup_datetime_2": "pickup_time",
        "dropoff_datetime_1": "dropoff_date",
        "dropoff_datetime_2": "dropoff_time"
    }))
renamed_col_df.head(5)

,vendor,pickup_datetime,pickup_date,pickup_time,dropoff_datetime,dropoff_date,dropoff_time,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [36]:
renamed_col_df.get_profile()


,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,7158.0,0.0,7158.0,0.0,0.0,0.0,5,,,,,,,,,,,,,,
pickup_datetime,FieldType.STRING,2009-01-01 20:52:58,2013-08-31 23:59:14,7158.0,0.0,7158.0,0.0,0.0,0.0,,,,,,,,,,,,,,,
pickup_date,FieldType.STRING,2009-01-01,2013-08-31,7158.0,0.0,7158.0,0.0,0.0,0.0,41,,,,,,,,,,,,,,
pickup_time,FieldType.STRING,00:00:48,23:59:48,7158.0,0.0,7158.0,0.0,0.0,0.0,,,,,,,,,,,,,,,
dropoff_datetime,FieldType.STRING,2009-01-01 21:14:00,2013-09-01 00:25:07,7158.0,0.0,7158.0,0.0,0.0,0.0,,,,,,,,,,,,,,,
dropoff_date,FieldType.STRING,2009-01-01,2013-09-01,7158.0,0.0,7158.0,0.0,0.0,0.0,42,,,,,,,,,,,,,,
dropoff_time,FieldType.STRING,00:00:00,23:59:45,7158.0,0.0,7158.0,0.0,0.0,0.0,,,,,,,,,,,,,,,
store_forward,FieldType.STRING,N,Y,7158.0,0.0,7158.0,0.0,0.0,0.0,2,,,,,,,,,,,,,,
pickup_longitude,FieldType.DECIMAL,-74.0782,-73.7365,7158.0,0.0,7158.0,0.0,0.0,0.0,,-74.078,-73.9639,-73.964,-73.9495,-73.9294,-73.8686,-73.8305,-73.8245,-73.7696,-73.9148,0.0490486,0.00240576,0.403432,-0.616452
pickup_latitude,FieldType.DECIMAL,40.5755,40.8799,7158.0,0.0,7158.0,0.0,0.0,0.0,,40.6334,40.7133,40.7131,40.7214,40.7581,40.805,40.8487,40.8675,40.8778,40.765,0.0481071,0.0023143,0.237329,-0.577294


In [37]:
transformed_features_df = (renamed_col_df
    .derive_column_by_example(
        source_columns="pickup_date",
        new_column_name="pickup_weekday",
        example_data=[("2009-01-04", "Sunday"), ("2013-08-22", "Thursday")]
    )
    .derive_column_by_example(
        source_columns="dropoff_date",
        new_column_name="dropoff_weekday",
        example_data=[("2013-08-22", "Thursday"), ("2013-11-03", "Sunday")]
    )

    .split_column_by_example(source_column="pickup_time")
    .split_column_by_example(source_column="dropoff_time")
    # The following two calls to split_column_by_example reference the column names generated from the previous two calls.
    .split_column_by_example(source_column="pickup_time_1")
    .split_column_by_example(source_column="dropoff_time_1")
    .drop_columns(columns=[
        "pickup_date", "pickup_time", "dropoff_date", "dropoff_time",
        "pickup_date_1", "dropoff_date_1", "pickup_time_1", "dropoff_time_1"
    ])

    .rename_columns(column_pairs={
        "pickup_date_2": "pickup_month",
        "pickup_date_3": "pickup_monthday",
        "pickup_time_1_1": "pickup_hour",
        "pickup_time_1_2": "pickup_minute",
        "pickup_time_2": "pickup_second",
        "dropoff_date_2": "dropoff_month",
        "dropoff_date_3": "dropoff_monthday",
        "dropoff_time_1_1": "dropoff_hour",
        "dropoff_time_1_2": "dropoff_minute",
        "dropoff_time_2": "dropoff_second"
    }))

transformed_features_df.head(5)

,vendor,pickup_datetime,pickup_weekday,pickup_hour,pickup_minute,pickup_second,dropoff_datetime,dropoff_weekday,dropoff_hour,dropoff_minute,dropoff_second,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,Thursday,17,22,00,2013-08-01 17:22:00,Thursday,17,22,00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,Thursday,17,24,00,2013-08-01 17:25:00,Thursday,17,25,00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,Tuesday,06,51,19,2013-08-06 06:51:36,Tuesday,06,51,36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,Tuesday,13,26,34,2013-08-06 13:26:57,Tuesday,13,26,57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,Tuesday,13,27,53,2013-08-06 13:28:08,Tuesday,13,28,08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [38]:
processed_df = transformed_features_df.drop_columns(columns=["pickup_datetime", "dropoff_datetime"])

In [39]:
type_infer = processed_df.builders.set_column_types()
type_infer.learn()
type_infer

Column types conversion candidates:
'pickup_weekday': [FieldType.STRING],
'pickup_hour': [FieldType.INTEGER],
'pickup_second': [FieldType.INTEGER],
'dropoff_minute': [FieldType.INTEGER],
'store_forward': [FieldType.BOOLEAN],
'pickup_minute': [FieldType.INTEGER],
'dropoff_weekday': [FieldType.STRING],
'vendor': [FieldType.STRING],
'dropoff_hour': [FieldType.INTEGER],
'dropoff_second': [FieldType.INTEGER],
'pickup_longitude': [FieldType.DECIMAL],
'pickup_latitude': [FieldType.DECIMAL],
'dropoff_longitude': [FieldType.DECIMAL],
'dropoff_latitude': [FieldType.DECIMAL],
'passengers': [FieldType.INTEGER],
'distance': [FieldType.DECIMAL],
'cost': [FieldType.DECIMAL]

In [40]:

type_converted_df = type_infer.to_dataflow()
type_converted_df.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,7158.0,0.0,7158.0,0.0,0.0,0.0,5,,,,,,,,,,,,,,
pickup_weekday,FieldType.STRING,Friday,Wednesday,7158.0,0.0,7158.0,0.0,0.0,0.0,7,,,,,,,,,,,,,,
pickup_hour,FieldType.INTEGER,0,23,7158.0,0.0,7158.0,0.0,0.0,0.0,24,0,3.66702,3,10,16,19,22.0838,23,23,14.2091,6.3535,40.3669,-0.68828,-0.469837
pickup_minute,FieldType.INTEGER,0,59,7158.0,0.0,7158.0,0.0,0.0,0.0,60,0,5,5,14,29.4629,44.8193,56.7194,59,59,29.4633,17.4448,304.322,0.00405393,-1.20535
pickup_second,FieldType.INTEGER,0,59,7158.0,0.0,7158.0,0.0,0.0,0.0,60,0,5,5,14.6446,29.9548,44.4496,56.9056,59,59,29.6105,17.3945,302.568,-0.0237901,-1.19458
dropoff_weekday,FieldType.STRING,Friday,Wednesday,7158.0,0.0,7158.0,0.0,0.0,0.0,7,,,,,,,,,,,,,,
dropoff_hour,FieldType.INTEGER,0,23,7158.0,0.0,7158.0,0.0,0.0,0.0,24,0,3,3,10,16,19,22,23,23,14.1583,6.45884,41.7166,-0.686119,-0.510164
dropoff_minute,FieldType.INTEGER,0,59,7158.0,0.0,7158.0,0.0,0.0,0.0,60,0,5.07264,5,14,28.9937,43.9915,56.9067,59,59,29.3236,17.4276,303.721,0.0170342,-1.21476
dropoff_second,FieldType.INTEGER,0,59,7158.0,0.0,7158.0,0.0,0.0,0.0,60,0,5,5,14.8215,29.7021,45.2322,56,59,59,29.7917,17.4891,305.869,-0.0311983,-1.21957
store_forward,FieldType.BOOLEAN,False,True,7158.0,0.0,7158.0,0.0,0.0,0.0,2,,,,,,,,,,,,,,


In [41]:
final_df = type_converted_df.filter(dprep.col("distance") > 0)
final_df = final_df.filter(dprep.col("cost") > 0)

In [42]:
import os
file_path = os.path.join(os.getcwd(), "dflows.dprep")

final_df.save(file_path)

# Data preprocesed
# ===============================
# AutoML start

In [43]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging
import os

In [44]:
ws = Workspace.from_config()
# choose a name for the run history container in the workspace
experiment_name = 'automated-ml-regression'
# project folder
project_folder = './automated-ml-regression'

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Location,canadacentral
Project Directory,./automated-ml-regression
Resource Group,ICSE-tutorial
SDK version,1.0.39
Subscription ID,942813ed-26a2-4790-9097-d4be7560fce0
Workspace,workspace_fresher


In [45]:
import azureml.dataprep as dprep

file_path = os.path.join(os.getcwd(), "dflows.dprep")

dflow_prepared = dprep.Dataflow.open(file_path)
dflow_prepared.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,6246.0,0.0,6246.0,0.0,0.0,0.0,5,,,,,,,,,,,,,,
pickup_weekday,FieldType.STRING,Friday,Wednesday,6246.0,0.0,6246.0,0.0,0.0,0.0,7,,,,,,,,,,,,,,
pickup_hour,FieldType.INTEGER,0,23,6246.0,0.0,6246.0,0.0,0.0,0.0,24,0,2.99166,2.87273,9.8659,16,19.2721,22.855,23,23,14.2453,6.59396,43.4803,-0.687416,-0.580976
pickup_minute,FieldType.INTEGER,0,59,6246.0,0.0,6246.0,0.0,0.0,0.0,60,0,5.10865,5,13.9974,29.2586,44.9126,56.6375,59,59,29.4236,17.4385,304.1,0.0118518,-1.21036
pickup_second,FieldType.INTEGER,0,59,6246.0,0.0,6246.0,0.0,0.0,0.0,60,0,5.32404,5,14.8348,29.9803,44.8506,56.9581,59,59,29.7273,17.3699,301.712,-0.0262142,-1.1968
dropoff_weekday,FieldType.STRING,Friday,Wednesday,6246.0,0.0,6246.0,0.0,0.0,0.0,7,,,,,,,,,,,,,,
dropoff_hour,FieldType.INTEGER,0,23,6246.0,0.0,6246.0,0.0,0.0,0.0,24,0,2.48966,2,9.50341,16,19.5775,22.9009,23,23,14.1832,6.71094,45.0367,-0.681267,-0.629212
dropoff_minute,FieldType.INTEGER,0,59,6246.0,0.0,6246.0,0.0,0.0,0.0,60,0,5.35778,5,14.0805,28.7827,43.9937,56.8706,59,59,29.2541,17.4137,303.238,0.0255643,-1.2171
dropoff_second,FieldType.INTEGER,0,59,6246.0,0.0,6246.0,0.0,0.0,0.0,60,0,5,5,14.6609,29.668,45.3462,56.291,59,59,29.7755,17.5407,307.675,-0.0250917,-1.22561
store_forward,FieldType.BOOLEAN,False,True,6246.0,0.0,6246.0,0.0,0.0,0.0,2,,,,,,,,,,,,,,


In [46]:

dflow_X = dflow_prepared.keep_columns(['pickup_weekday','pickup_hour', 'distance','passengers', 'vendor'])
dflow_y = dflow_prepared.keep_columns('cost')

In [47]:
from sklearn.model_selection import train_test_split


x_df = dflow_X.to_pandas_dataframe()
y_df = dflow_y.to_pandas_dataframe()

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)
# flatten y_train to 1d array
y_train.values.flatten()

array([ 5.5, 10.5,  8. , ..., 11. , 33.5,  6.7])

In [48]:
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 30,
    "primary_metric" : 'spearman_correlation',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

In [49]:
from azureml.train.automl import AutoMLConfig

# local compute 
automated_ml_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automated_ml_errors.log',
                             path = project_folder,
                             X = x_train.values,
                             y = y_train.values.flatten(),
                             **automl_settings)

In [ ]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_c12516df-c745-4f1e-912d-bb0f29224047
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION 

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()